In [28]:
import os, sys, json
from collections import Counter
import importlib
import random
from copy import deepcopy

random.seed(0)

/home/anmol/nips_challenge/efficiency_challenge_repo/external_repos/helm/src/helm/benchmark/scenarios/bbq_scenario.py

#### Used in:
* Accuracy

In [29]:
sys.path.append('/home/anmol/nips_challenge/efficiency_challenge_repo/external_repos/helm/src/helm/benchmark/scenarios')
sys.path.append('/home/anmol/nips_challenge/efficiency_challenge_repo/external_repos/helm/src/helm/benchmark/augmentations/')

In [30]:
sys.path.append('../')

In [31]:
import utils_code.helper as helper
importlib.reload(helper)

<module 'utils_code.helper' from '/home/anmol/nips_challenge/efficiency_challenge_repo/code/02_dataset_curation/../utils_code/helper.py'>

In [32]:
import helm
from helm.benchmark.scenarios.bbq_scenario import BBQScenario
from helm.benchmark.scenarios.truthful_qa_scenario import TruthfulQAScenario
from helm.benchmark.scenarios.mmlu_scenario import MMLUScenario
from helm.benchmark.scenarios.summarization_scenario import SummarizationScenario
from helm.benchmark.scenarios.big_bench_scenario import BIGBenchScenario


from helm.benchmark.scenarios.scenario import Scenario, ScenarioSpec


from helm.benchmark.augmentations.mild_mix_perturbation import MildMixPerturbation
from helm.benchmark.augmentations.dialect_perturbation import DialectPerturbation
from helm.benchmark.augmentations.gender_perturbation import GenderPerturbation
from helm.benchmark.augmentations.person_name_perturbation import PersonNamePerturbation



####  BigBench  dataset

In [33]:
bigbench_tasks = {
    "analytic_entailment":[],
"causal_judgment":[],
"emoji_movie":[],
"empirical_judgments":[],
"known_unknowns":[],
"logical_deduction":["three_objects", "five_objects", "seven_objects"],
"strange_stories":["multiple_choice", "boolean"],
#"logical_deduction":["three_objects"],
#"strange_stories":["multiple_choice"],
"snarks":[],
"dark_humor_detection":[]
}

In [34]:
extra_tasks_dict = {
    "causal_judgment": [],
    "cause_and_effect": [
                'one_sentence'
              ],
    "com2sense": [],
    "empirical_judgments": [],
    "entailed_polarity": [],
    #"entailed_polarity_hindi": [],
    "fantasy_reasoning": [],
    "figure_of_speech_detection": [],
    "forecasting_subquestions": [],
    #"goal_step_wikihow": [],
    #"human_organs_senses": [],
    #"indic_cause_and_effect": [],
    "minute_mysteries_qa": [],
    "moral_permissibility": [],
    "simple_ethical_questions": [],
    "tellmewhy": [],
    #"winowhy": [],
    "analytic_entailment": [],
    "auto_categorization": [],
    #"auto_debugging": [],
    "boolean_expressions": [],
    #"checkmate_in_one": [],
    #"chess_state_tracking": [],
    "code_line_description": [],
    "codenames": [],
    #"conlang_translation": [],
    "context_definition_alignment": [],
    #"crass_ai": [],
    "cryptonite": [],
    "date_understanding": [],
    #"dyck_languages": [],
    #"dynamic_counting": [],
    #"elementary_math_qa": [],
    "epistemic_reasoning": [],
    #"evaluating_information_essentiality": [],
    "formal_fallacies_syllogisms_negation": [],
    #"gender_sensitivity_chinese": [],
    "gender_sensitivity_english": [],
    #"gre_reading_comprehension": [],
    #"identify_math_theorems": [],
    #"intersect_geometry": [],
    #"kannada": [],
    #"key_value_maps": [],
    "language_games": [],
    #"linguistics_puzzles": [],
    "logic_grid_puzzle": [],
    #"logical_args": [],
    "logical_deduction": [],
    "logical_fallacy_detection": [],
    #"navigate": [],
    #"nonsense_words_grammar": [],
    "object_counting": [],
    #"penguins_in_a_table": [],
    #"physics_questions": [],
    "presuppositions_as_nli": [],
    #"program_synthesis": [],
    "reasoning_about_colored_objects": [],
    #"repeat_copy_logic": [],
    #"rephrase": [],
    #"roots_optimization_and_games": [],
    "simp_turing_concept": [],
    #"simple_text_editing": [],
    "spelling_bee": [],
    #"strategyqa": [],
    "sudoku": [],
    #"sufficient_information": [],
    #"symbol_interpretation": [],
    #"temporal_sequences": [],
    #"timedial": [],
    "tracking_shuffled_objects": [],
    #"undo_permutation": [],
    #"unit_interpretation": [],
    #"web_of_lies": [],
    #"word_problems_on_sets_and_graphs": []
}
_keys = list(extra_tasks_dict.keys())
for _key in _keys:
    if _key in bigbench_tasks:
        del extra_tasks_dict[_key]

In [35]:
!ls ./

00_dataset_curation.ipynb		       benchmark_output
01_helm_train_split_dataset_curation.ipynb     bigbench_scenario.py
02_helm_train_split_bbq.ipynb		       data
03_helm_train_split_mmlu.ipynb		       data.lock
04_helm_train_split_truthfulqa.ipynb	       gsm_data_test
05_helm_train_split_cnndm.ipynb		       gsm_data_test.lock
06_helm_train_split_bigbench.ipynb	       gsm_data_train
07_make_combined_dataset.ipynb		       gsm_data_train.lock
08_helm_train_split_bb_stem_moral_tasks.ipynb  new_math_tasks.txt
09_helm_train_split_gsm8k.ipynb		       plot_curves.ipynb
10_helm_train_split_mathqa.ipynb	       word_list_freq.json


In [36]:
extra_tasks_dict = \
{'physics': [''],
 #'unnatural_in_context_learning': [''],
 #'evaluating_information_essentiality': [''],
 #'arithmetic': [''],
 #'long_context_integration': [''],
 'elementary_math_qa': ['question_with_language_hint', 'question_only','mathematical_hint_only','language_hint_only',\
                       'question_with_mathematical_hint'],
 #'intersect_geometry': [''],
 #'geometric_shapes': [''],
 'simple_arithmetic_json_multiple_choice': [''],
 'physics_questions': [''],
 'physical_intuition': [''],
 'mathematical_induction': [''],
 #'protein_interacting_sites': [''],
 #'simple_arithmetic': [''],
 #'key_value_maps': [''],
 #'identify_math_theorems': [''],
 'chinese_remainder_theorem': [''],
 'navigate': [''],
 'simple_arithmetic_json_subtasks': ['one_digit','two_digit','three_digit'],
 #'operators': [''],
 'simple_arithmetic_json': [''],
 #'dynamic_counting': [''],
 #'program_synthesis': [''],
 'unit_interpretation': ['lv0', 'lv1', 'lv2', 'lv3'],
 'auto_debugging': [''],
 #'periodic_elements': ['subtask_1'],
 'scientific_press_release': [''],
 #'roots_optimization_and_games': [''],
 #'multistep_arithmetic': [''],
 #'checkmate_in_one': [''],
 #'repeat_copy_logic': [''],
 'unit_conversion': ['unit_identification', 'different_systems'],
 #'simple_text_editing': [''],
 #'dyck_languages': [''],
 #'cryobiology_spanish': [''],
 'simple_arithmetic_multiple_targets_json': [''],
 'matrixshapes': [''],
 'vitaminc_fact_verification': [''],
 #'modified_arithmetic': ['']
}

In [37]:

ans_df = dict()


In [38]:
dict_to_use = extra_tasks_dict
for curr_task in dict_to_use:
    print("##########")
    print("TASK is: ", curr_task)
    try:
        if len(dict_to_use[curr_task])==0:
            dict_to_use[curr_task]=  [""]
        for subtask in dict_to_use[curr_task]:
            big_bench_obj = BIGBenchScenario(curr_task, subtask)
            if subtask == "":
                big_bench_obj.output_path = f"../01_helm_eval_setup/benchmark_output/scenarios/big_bench/{curr_task}"
            else:
                big_bench_obj.output_path = f"../01_helm_eval_setup/benchmark_output/scenarios/big_bench/{curr_task}/{subtask}"
            ans = big_bench_obj.get_instances(big_bench_obj.output_path)
            ans = list(filter(lambda x:helper.fetch_num_correct_options(x)==1, ans))
            random.shuffle(ans)
            if subtask == "":       
                _tmp_train = list(filter(lambda x:x.split=='train', ans))[:300]
                _tmp_valid = list(filter(lambda x:x.split=='valid', ans))[:50]
                _tmp_test = list(filter(lambda x:x.split=='test', ans))[:50]
            else:
                _tmp_train = list(filter(lambda x:x.split=='train', ans))[:80]
                _tmp_valid = list(filter(lambda x:x.split=='valid', ans))[:10]
                _tmp_test = list(filter(lambda x:x.split=='test', ans))[:10]
                '''if len(_tmp_train) == 0:
                    _tmp_train = _tmp_valid
                    for _elem in _tmp_train:
                        _elem.split = "train"
                    _tmp_valid = []'''
            
            ans = _tmp_train + _tmp_valid + _tmp_test
            _tag  = curr_task+"|"+subtask
            ans_df[_tag] = ans
            print("TRAIN-TEST SPLIT are: ", Counter([x.split for x in ans]))
    except Exception as E:
        print("Error is: ", E)

##########
TASK is:  physics
ensure_file_downloaded {
  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/physics/task.json because ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/physics/physics/task.json already exists
} [0.001s]
TRAIN-TEST SPLIT are:  Counter({'train': 139, 'valid': 45, 'test': 45})
##########
TASK is:  elementary_math_qa
ensure_file_downloaded {
  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/elementary_math_qa/question_with_language_hint/task.json because ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/elementary_math_qa/question_with_language_hint/elementary_math_qa/question_with_language_hint/task.json already exists
} [0.0s]
TRAIN-TEST SPLIT are:  Counter({'train': 80, 'valid': 10, 'test': 10})
ensure_file_downloaded {
  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/elementary_math_qa/question

  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/auto_debugging/task.json because ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/auto_debugging/auto_debugging/task.json already exists
} [0.0s]
TRAIN-TEST SPLIT are:  Counter({'test': 14, 'valid': 13, 'train': 2})
##########
TASK is:  scientific_press_release
ensure_file_downloaded {
  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/scientific_press_release/task.json because ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/scientific_press_release/scientific_press_release/task.json already exists
} [0.0s]
TRAIN-TEST SPLIT are:  Counter({'train': 18, 'valid': 16, 'test': 16})
##########
TASK is:  unit_conversion
ensure_file_downloaded {
  Not downloading https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/unit_conversion/unit_identification/task.json because ../01_helm_eval_setup/benchmark_o

In [39]:
for curr_key, curr_val in ans_df.items():
    print("Key: ", curr_key, " | Split: ", Counter([x.split for x in curr_val]) )
    #print(curr_val[0].input.text)

Key:  physics|  | Split:  Counter({'train': 139, 'valid': 45, 'test': 45})
Key:  elementary_math_qa|question_with_language_hint  | Split:  Counter({'train': 80, 'valid': 10, 'test': 10})
Key:  elementary_math_qa|question_only  | Split:  Counter({'train': 80, 'valid': 10, 'test': 10})
Key:  elementary_math_qa|mathematical_hint_only  | Split:  Counter({'train': 80, 'valid': 10, 'test': 10})
Key:  elementary_math_qa|language_hint_only  | Split:  Counter({'train': 80, 'valid': 10, 'test': 10})
Key:  elementary_math_qa|question_with_mathematical_hint  | Split:  Counter({'train': 80, 'valid': 10, 'test': 10})
Key:  simple_arithmetic_json_multiple_choice|  | Split:  Counter({'valid': 8})
Key:  physics_questions|  | Split:  Counter({'train': 11, 'valid': 6, 'test': 4})
Key:  physical_intuition|  | Split:  Counter({'train': 49, 'valid': 16, 'test': 16})
Key:  mathematical_induction|  | Split:  Counter({'train': 38, 'valid': 16, 'test': 16})
Key:  chinese_remainder_theorem|  | Split:  Counter({'

```python
adapter_spec = AdapterSpec(
        method=get_adaptation_method(big_bench_task["metrics"]),
        model="openai/text-curie-001",  # Can override with the `ModelRunExpander`.
        max_train_instances=5,  # Can override with the `MaxTrainInstancesRunExpander`.
        num_outputs=1,  # Can override with the `NumOutputsRunExpander`.
        # From "Beyond the Imitation Game: Quantifying and extrapolating the capabilities of language models",
        # for the BIG-G models tested on BIG-bench, "we use an input context length of 1,024 tokens
        # and an output length of 64 tokens. We evaluate on up to 1,000 examples per task".
        max_tokens=64,
        # "all model outputs were sampled greedily (with zero temperature), unless otherwise noted."
        temperature=0,
        instructions=big_bench_task.get("task_prefix", ""),
        # BIG-bench's default value for "example_input_prefix" and "example_output_prefix" was "\nQ: " and "\nA: ".
        # Instead, use our defaults for multiple choice tasks: "Question: " and "\nAnswer: ".
        input_prefix=big_bench_task.get("example_input_prefix", "Question: "),
        output_prefix=big_bench_task.get("example_output_prefix", "Answer: "),
        # Use our default for multiple choice: A., B., C., D.,...
        # reference_prefix=big_bench_task.get("choice_prefix", "\n choice: "),
        # The default value for "stop_string" in BIG-bench is None.
        stop_sequences=[str(big_bench_task.get("stop_string"))] if big_bench_task.get("stop_string", None) else [],
    )
```

In [40]:
BASE_DIR = "../01_helm_eval_setup/benchmark_output/scenarios/big_bench"

In [41]:
whole_arr = []

bb_id = -1
for curr_section in ans_df:
    print("#################")
    print("------- " , curr_section, " --------- ")
    curr_task, curr_subtask = curr_section.split("|")
    if curr_subtask == "":
        path_to_read = os.path.join(BASE_DIR, curr_task, "task.json")
    else:
        path_to_read = os.path.join(BASE_DIR, curr_task, curr_subtask, curr_task, curr_subtask, "task.json")
    if not os.path.exists(path_to_read):
        path_to_read = os.path.join(BASE_DIR, curr_task, curr_task,  "task.json")
    print(curr_task, curr_subtask, path_to_read)
    try:
        with open(path_to_read, 'r') as fd:
            inst_df = json.load(fd)
    except:
        print("SKIPPING.")
        print(path_to_read)
        continue
    ###########################
    instructions=inst_df.get("task_prefix", "")
    input_prefix=inst_df.get("example_input_prefix", "Question: ")
    output_prefix=inst_df.get("example_output_prefix", "Answer: ")
    ############################
    for _eg in ans_df[curr_section]:
        random.shuffle(_eg.__dict__['references'])
        bb_id+=1
        obj = dict()
        obj['bb_stem_id'] = bb_id
        obj['section'] = curr_section
        obj['task'] = curr_task
        obj['subtask'] = curr_subtask
        obj['org_task'] = True if curr_task in bigbench_tasks else False
        obj['split_used'] = _eg.split
        input_str, correct_option = helper.fetch_general_task(_eg, instructions, input_prefix, output_prefix)
        obj['instruction'] = input_str
        obj['output'] = correct_option
        whole_arr.append(obj)
        #break
    print(whole_arr[-1]['instruction'])
    print("OUTPUT:::", whole_arr[-1]['output'])
    #print('$$$$$')
    #print(whole_arr[-2]['instruction'])
    #print("OUTPUT:::", whole_arr[-2]['output'])
    #break
    #if curr_section == ""

#################
-------  physics|  --------- 
physics  ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/physics/physics/task.json
Identify the physics formula that would be most useful for finding the answer to each of the following word problems.

Question: A car sits in an entrance ramp to a freeway, waiting for a break in traffic. The driver sees a small gap between two vehicles and accelerates with constant acceleration along the ramp onto the freeway. The car starts from rest, moves in a straight line, and has a speed of 20 m/s when it reaches the end of the 120-m ramp. How much time does it take the car to reach the end of the ramp?
A. p = m * v
B. dx = v * dt
C. F = dp / dt
D. v = v_0 + a * t
Answer: 
OUTPUT::: D
#################
-------  elementary_math_qa|question_with_language_hint  --------- 
elementary_math_qa question_with_language_hint ../01_helm_eval_setup/benchmark_output/scenarios/big_bench/elementary_math_qa/question_with_language_hint/elementary_math_qa/

Based only on the information contained in a brief quote from Wikipedia, answer whether the related claim is True, False or Neither. Use Neither when the Wikipedia quote does not provide the necessary information to resolve the question.


Passage: County Durham: Other large settlements in the County include Durham , Bishop Auckland , Consett , Crook , Easington , Newton Aycliffe , Hartlepool , Spennymoor , Ferryhill , Stockton-on-Tees .
Claim: Crook , Eastington , Ferryhill , and Stockton-on-Tees are settlements in County Durham .
A. Neither
B. False
C. True

True, False, or Neither? 
OUTPUT::: C


In [42]:
bb_id

2927

In [43]:
output_prefix

'\nTrue, False, or Neither? '

In [44]:
helper.fetch_num_correct_options(_eg)

1

In [45]:
len(whole_arr)

2928

In [46]:
Counter([x['split_used'] for x in whole_arr])

Counter({'train': 2017, 'valid': 480, 'test': 431})

In [47]:
print(whole_arr[0]['instruction'])

Identify the physics formula that would be most useful for finding the answer to each of the following word problems.

Question: A cart is initially moving at 0.5 m/s along a track. The cart comes to rest after traveling 1 m. The experiment is repeated on the same track, but now the cart is initially moving at 1 m/s. How far does the cart travel before coming to rest?
A. dE = k * λ * dx / x ^ 2
B. v = dx / dt
C. v ^ 2 = v_0 ^ 2 + 2 * a * d
D. E = k * q / r ^ 2
Answer: 


In [48]:
train_arr = list(filter(lambda x: "train" in x['split_used'], whole_arr))
valid_arr = list(filter(lambda x: "valid" == x['split_used'], whole_arr))
test_arr = list(filter(lambda x: "test" == x['split_used'], whole_arr))

In [49]:
train_arr[0]

{'bb_stem_id': 0,
 'section': 'physics|',
 'task': 'physics',
 'subtask': '',
 'org_task': False,
 'split_used': 'train',
 'instruction': 'Identify the physics formula that would be most useful for finding the answer to each of the following word problems.\n\nQuestion: A cart is initially moving at 0.5 m/s along a track. The cart comes to rest after traveling 1 m. The experiment is repeated on the same track, but now the cart is initially moving at 1 m/s. How far does the cart travel before coming to rest?\nA. dE = k * λ * dx / x ^ 2\nB. v = dx / dt\nC. v ^ 2 = v_0 ^ 2 + 2 * a * d\nD. E = k * q / r ^ 2\nAnswer: ',
 'output': 'B'}

In [50]:
arr_look = train_arr
print(len(arr_look))
print(Counter([x['split_used'] for x in arr_look]))
#print(Counter([x['output'] for x in arr_look]))
print(Counter([x['section'] for x in arr_look]))
#print(Counter([x['perturb_type'] for x in arr_look]))

2017
Counter({'train': 2017})
Counter({'chinese_remainder_theorem|': 300, 'navigate|': 300, 'matrixshapes|': 300, 'vitaminc_fact_verification|': 300, 'physics|': 139, 'elementary_math_qa|question_with_language_hint': 80, 'elementary_math_qa|question_only': 80, 'elementary_math_qa|mathematical_hint_only': 80, 'elementary_math_qa|language_hint_only': 80, 'elementary_math_qa|question_with_mathematical_hint': 80, 'unit_conversion|unit_identification': 80, 'unit_conversion|different_systems': 80, 'physical_intuition|': 49, 'mathematical_induction|': 38, 'scientific_press_release|': 18, 'physics_questions|': 11, 'auto_debugging|': 2})


In [51]:
arr_look = valid_arr
print(len(arr_look))
print(Counter([x['split_used'] for x in arr_look]))
#print(Counter([x['output'] for x in arr_look]))
print(Counter([x['section'] for x in arr_look]))

#print(Counter([x['perturb_type'] for x in arr_look]))

480
Counter({'valid': 480})
Counter({'chinese_remainder_theorem|': 50, 'navigate|': 50, 'matrixshapes|': 50, 'vitaminc_fact_verification|': 50, 'physics|': 45, 'physical_intuition|': 16, 'mathematical_induction|': 16, 'simple_arithmetic_json|': 16, 'scientific_press_release|': 16, 'auto_debugging|': 13, 'elementary_math_qa|question_with_language_hint': 10, 'elementary_math_qa|question_only': 10, 'elementary_math_qa|mathematical_hint_only': 10, 'elementary_math_qa|language_hint_only': 10, 'elementary_math_qa|question_with_mathematical_hint': 10, 'simple_arithmetic_json_subtasks|one_digit': 10, 'simple_arithmetic_json_subtasks|two_digit': 10, 'simple_arithmetic_json_subtasks|three_digit': 10, 'unit_interpretation|lv0': 10, 'unit_interpretation|lv1': 10, 'unit_interpretation|lv2': 10, 'unit_interpretation|lv3': 10, 'unit_conversion|unit_identification': 10, 'unit_conversion|different_systems': 10, 'simple_arithmetic_json_multiple_choice|': 8, 'physics_questions|': 6, 'simple_arithmetic_mu

In [52]:
arr_look = test_arr
print(len(arr_look))
print(Counter([x['split_used'] for x in arr_look]))
#print(Counter([x['output'] for x in arr_look]))
#print(Counter([x['perturb_type'] for x in arr_look]))
print(Counter([x['section'] for x in arr_look]))


431
Counter({'test': 431})
Counter({'chinese_remainder_theorem|': 50, 'navigate|': 50, 'matrixshapes|': 50, 'vitaminc_fact_verification|': 50, 'physics|': 45, 'physical_intuition|': 16, 'mathematical_induction|': 16, 'scientific_press_release|': 16, 'simple_arithmetic_json|': 14, 'auto_debugging|': 14, 'elementary_math_qa|question_with_language_hint': 10, 'elementary_math_qa|question_only': 10, 'elementary_math_qa|mathematical_hint_only': 10, 'elementary_math_qa|language_hint_only': 10, 'elementary_math_qa|question_with_mathematical_hint': 10, 'unit_conversion|unit_identification': 10, 'unit_conversion|different_systems': 10, 'unit_interpretation|lv0': 9, 'unit_interpretation|lv1': 9, 'unit_interpretation|lv2': 9, 'unit_interpretation|lv3': 9, 'physics_questions|': 4})


In [53]:
random.shuffle(train_arr)
random.shuffle(valid_arr)
random.shuffle(test_arr)

In [54]:


save_path  = "../../data/new_training_datasets/pegasus_stem_bigbench_train_dataset.json"
with open(save_path, 'w') as fd:
    json.dump(train_arr, fd, indent=1)
    
save_path  = "../../data/new_training_datasets/pegasus_stem_bigbench_valid_dataset.json"
with open(save_path, 'w') as fd:
    json.dump(valid_arr, fd, indent=1)
    
save_path  = "../../data/new_training_datasets/pegasus_stem_bigbench_test_dataset.json"
with open(save_path, 'w') as fd:
    json.dump(test_arr, fd, indent=1)